In [8]:
import os
from pathlib import Path
import pandas as pd
import json
from epjson.epjson_idf import EpJsonIDF
from epjson.simulate_cluster import SimulateCluster

def run_single_building(
        config_dict, 
        output_dir, 
        weight_map,
        resource_dir=Path("tests/resources"), 
        idf_dir=Path("tests/resources/idfs/"),
        eplus_loc=Path("C:\EnergyPlusV22-2-0"), 
        update_epsjon=True
        ):
    
    weights_df = pd.read_json(resource_dir / "shoebox-weights.json")
    weights_df = weights_df[weights_df["building_id"] == config_dict["building_id"]]
    idfs = set(weights_df.ShoeboxPath.to_list())

    ### MUTATIONS ###

    # Get retrofit scenario template data
    features_df = pd.read_csv(resource_dir / f"templates_{config_dict['retrofit_scenario']}.csv")

    with open(resource_dir / "template_features_map.json", "r") as f:
        features_map = json.load(f)
    with open(resource_dir / "schedule_bunches.json", "r") as f:
        schedules = json.load(f)
        
    if update_epsjon:
        for idf in idfs:
            try:
                idf_path = idf_dir / idf
                epjson = EpJsonIDF.from_epjson(idf_path.with_suffix(".epjson"), eplus_loc=eplus_loc)
                features_series = features_df[(features_df.TYPOLOGY == config_dict["TYPOLOGY"]) & (features_df.AGE_ERA == config_dict["AGE_ERA"])].reset_index(drop=True)
                features_series = features_series.iloc[0]

                cols = set(features_series.index) & set(features_map.keys())

                epjson.replace_features(features_series[cols], features_map)
                epjson.update_schedules(schedules)
                epjson.add([(
                    "Output:SQLite",
                    {"Output:SQLite 1": {"option_type": "Simple"}}
                    )])
                epjson.epjson["ZoneInfiltration:DesignFlowRate"]["CoreInfiltration"].update(
                    {"air_changes_per_hour": 0}
                )
                
                # new_path = replace_shoebox_path(idf_path).with_suffix(".epJSON")
                new_path = output_dir / Path(idf).with_suffix(".epJSON")
                epjson.save_as(new_path)
                epjson.save_idf(output_path=new_path.parent) #suffix="_test",

            except Exception as e:
                print(f"FAILED ON {idf}")
                raise e
    
    ### SIMULATION ###
    # Get new paths for updated shoeboxes
    # new_idfs = weights_df["ShoeboxPath"].apply(replace_shoebox_path)
    new_idfs = weights_df["ShoeboxPath"].apply(lambda x: output_dir / Path(x))
    weights_df["ShoeboxPath"] = new_idfs
    
    print("BUILDING CLUSTER")
    idf_cluster = SimulateCluster(
        idf_list = set(new_idfs),
        epw = config_dict["epw"],
        weights_df = weights_df,
        building_col = "building_id",
        eplus_location=eplus_loc,
        weight_map = weight_map
    )
    print("SIMULATING CLUSTER")
    errors = idf_cluster.parallel_simulate() # Fix errors dict

    print("PROCESSING CLUSTER RESULTS")
    ### POST-PROCESS ###
    r = idf_cluster.fetch_building_results_parallel([config_dict["building_id"]])

    return r, errors, idf_cluster

In [ ]:
config_dict = {
    "building_id": "2",
    "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
    "retrofit_scenario": "baseline",
    "schedules_scenaro": "baseline",
    "lab_scenario": "baseline",
    "TYPOLOGY": "Residential",
    "AGE_ERA": 1980,
    "lab_weight": 0.8
}


sb_weight_map = {
    "Perimeter": "PerimeterAreaWeight",
    "Core": "CoreAreaWeight"
}

office_res, office_err, office_cluster = run_single_building(config_dict, 
                               weight_map = sb_weight_map,
                               output_dir = Path(os.getcwd()) / "eplus", 
                               idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                               update_epsjon=True
                               )
office_res['2'].sum()

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
res = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "baseline",
        "schedules_scenaro": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    res[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [46]:
norm_cols = ['Zone Lights Electricity Energy_norm',
       'Electric Equipment Electricity Energy_norm',
       'Zone Ideal Loads Supply Air Total Heating Energy_norm',
       'Zone Ideal Loads Supply Air Total Cooling Energy_norm',
       'Water Use Equipment Heating Energy_norm']
pd.DataFrame.from_dict(res).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                            174.709437   
                     1980 W31                           152.870765   
Lab and Mixed Use    1945 1                             104.825664   
                     1980 13                            104.825662   
                     2015 12                            104.825661   
Office and Mixed Use 1945 50                            104.825659   
                     1980 W11                            50.665741   
Residential          1945 62                              31.75643   
                     1980 NW30                           31.756433   
                     2015 E37                            25.405145   
Support Dominant     1945 42                             26.570394   
                     1980 E23                            26.570394   
                     2015 42C                            26.570393   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   415.191115   
                     1980 W31                                  148.299103   
Lab and Mixed Use    1945 1                                     264.84174   
                     1980 13                                   124.743663   
                     2015 12                                    32.231307   
Office and Mixed Use 1945 50                                    174.68143   
                     1980 W11                                  209.858257   
Residential          1945 62                                    34.503715   
                     1980 NW30                                  19.112444   
                     2015 E37                                   18.329249   
Support Dominant     1945 42                                    70.262852   
                     1980 E23                                   68.219383   
                     2015 42C                                   57.525876   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                             1.197139      
                     1980 W31                                            0.002172      
Lab and Mixed Use    1945 1                                            168.294863      
                     1980 13                                              77.9282      
                     2015 12                                            94.337992      
Office and Mixed Use 1945 50                                           170.534793      
                     1980 W11                                          147.015152      
Residential          1945 62                                           121.598949      
                     1980 NW30                                          87.207127      
                     2015 E37                                           61.884706      
Support Dominant     1945 42                                            82.107743      
                     1980 E23                                            90.43698      
                     2015 42C                                           63.320952      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                           109.687613      
                     1980 W31                                           57.354087      
Lab and Mixed Use    1945 1                                            256.257825      
                     1980 13                                           205.277907      
                     2015 12                                           153.509578      
Office and Mixed Use 1945 50                                           210.562361      
                     1980 W11                                           158.66663      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
partialres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "partial",
        "schedules_scenaro": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    partialres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [47]:
pd.DataFrame.from_dict(partialres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              52.412832   
                     1980 13                             52.412831   
                     2015 12                              52.41283   
Office and Mixed Use 1945 50                              52.41283   
                     1980 W11                             25.33287   
Residential          1945 62                             15.878215   
                     1980 NW30                           15.878217   
                     2015 E37                            12.702573   
Support Dominant     1945 42                             13.285197   
                     1980 E23                            13.285197   
                     2015 42C                            13.285196   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   415.191115   
                     1980 W31                                  148.299103   
Lab and Mixed Use    1945 1                                     264.84174   
                     1980 13                                   124.743663   
                     2015 12                                    32.231307   
Office and Mixed Use 1945 50                                    174.68143   
                     1980 W11                                  209.858257   
Residential          1945 62                                    34.503715   
                     1980 NW30                                  19.112444   
                     2015 E37                                   18.329249   
Support Dominant     1945 42                                    70.262852   
                     1980 E23                                   68.219383   
                     2015 42C                                   57.525876   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                             0.997209      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                              1.810529      
                     1980 13                                             0.901098      
                     2015 12                                             9.142799      
Office and Mixed Use 1945 50                                             1.257335      
                     1980 W11                                           12.249601      
Residential          1945 62                                            89.697918      
                     1980 NW30                                          57.885413      
                     2015 E37                                           51.288745      
Support Dominant     1945 42                                            20.386046      
                     1980 E23                                           23.548917      
                     2015 42C                                           10.870339      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            62.361488      
                     1980 W31                                            28.60431      
Lab and Mixed Use    1945 1                                             71.178031      
                     1980 13                                            52.106076      
                     2015 12                                            43.359333      
Office and Mixed Use 1945 50                                            64.410787      
                     1980 W11                                           41.647905      
Residential          1945 62      

In [ ]:
sbs = pd.read_json("tests/resources/shoebox-weights.json")
sbs = sbs.groupby(["TYPOLOGY", "AGE_ERA"]).first().reset_index()
fullres = {}
for i, row in sbs.iterrows():
    config_dict = {
        "building_id": row.building_id,
        "epw": "D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\epws\\USA_MA_Boston-Logan.Intl.AP.725090_TMY3.epw",
        "retrofit_scenario": "full",
        "schedules_scenaro": "baseline",
        "lab_scenario": "baseline",
        "TYPOLOGY": row.TYPOLOGY,
        "AGE_ERA": row.AGE_ERA,
        "lab_weight": 0.8
    }
    office_res, office_err, office_cluster = run_single_building(config_dict, 
                                weight_map = sb_weight_map,
                                output_dir = Path(os.getcwd()) / "eplus", 
                                idf_dir=Path("D:\\Users\\zoelh\\Dropbox (MIT)\\4.S42 Campus Decarb\\Energy Modeling\\idfs\\gis\\simple_shoeboxes"),
                                update_epsjon=True
                                )
    print(row.TYPOLOGY, row.AGE_ERA)
    fullres[(row.TYPOLOGY, row.AGE_ERA, row.building_id)] = office_res[row.building_id].sum()
    

In [48]:
pd.DataFrame.from_dict(fullres).T[norm_cols]

Zone Lights Electricity Energy_norm  \
Athletic             1945 51                             87.354719   
                     1980 W31                             78.61925   
Lab and Mixed Use    1945 1                              52.412832   
                     1980 13                             52.412831   
                     2015 12                              52.41283   
Office and Mixed Use 1945 50                              52.41283   
                     1980 W11                             25.33287   
Residential          1945 62                             15.878215   
                     1980 NW30                           15.878217   
                     2015 E37                            12.702573   
Support Dominant     1945 42                             13.285197   
                     1980 E23                            13.285197   
                     2015 42C                            13.285196   

                               Electric Equipment Electricity Energy_norm  \
Athletic             1945 51                                   415.191115   
                     1980 W31                                  148.299103   
Lab and Mixed Use    1945 1                                     264.84174   
                     1980 13                                   124.743663   
                     2015 12                                    32.231307   
Office and Mixed Use 1945 50                                    174.68143   
                     1980 W11                                  209.858257   
Residential          1945 62                                    34.503715   
                     1980 NW30                                  19.112444   
                     2015 E37                                   18.329249   
Support Dominant     1945 42                                    70.262852   
                     1980 E23                                   68.219383   
                     2015 42C                                   57.525876   

                               Zone Ideal Loads Supply Air Total Heating Energy_norm  \
Athletic             1945 51                                             0.001646      
                     1980 W31                                                 0.0      
Lab and Mixed Use    1945 1                                              0.006114      
                     1980 13                                             0.006931      
                     2015 12                                             4.451708      
Office and Mixed Use 1945 50                                             0.003937      
                     1980 W11                                            3.543808      
Residential          1945 62                                            49.854151      
                     1980 NW30                                          31.284803      
                     2015 E37                                           35.363447      
Support Dominant     1945 42                                              6.97332      
                     1980 E23                                            6.070503      
                     2015 42C                                            4.085313      

                               Zone Ideal Loads Supply Air Total Cooling Energy_norm  \
Athletic             1945 51                                            61.501484      
                     1980 W31                                           29.169963      
Lab and Mixed Use    1945 1                                             73.490679      
                     1980 13                                            52.863584      
                     2015 12                                            44.248006      
Office and Mixed Use 1945 50                                              65.6316      
                     1980 W11                                           43.546319      
Residential          1945 62      